# 4.4.3. Limit between detection and classification

Simulation based on synthetic data.

In [20]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [5]:
from metrics.classification import MCC, SPEc
from metrics.detection import precision, recall, f1, average_precision

In [17]:
def _rand_clip(loc, scale=0.1):
    return np.clip(np.random.normal(loc=loc, scale=scale), 0, 1)

class Scenario:
    def __init__(self, 
                trivial_false,
                easy_false,
                hard_false,
                hard_true,
                easy_true,
                missed_true=0):
        self.trivial_false = trivial_false
        self.easy_false = easy_false
        self.hard_false = hard_false
        self.hard_true = hard_true
        self.easy_true = easy_true
        self.missed_true = missed_true
    
    def random_draw(self):
        false_candidates = [0 for _ in range(self.trivial_false)] + \
                           [_rand_clip(0.1, 0.1) for _ in range(self.easy_false)] + \
                           [_rand_clip(0.45, 0.1) for _ in range(self.hard_false)]
        true_candidates =  [_rand_clip(0.55, 0.1) for _ in range(self.hard_true)] + \
                           [_rand_clip(0.8, 0.05) for _ in range(self.easy_true)]
        
        gt = np.array([False for _ in false_candidates]+
                           [True for _ in true_candidates]+
                           [True for _ in range(self.missed_true)])
        pred = np.array(
                            [k for k in false_candidates] +
                            [k for k in true_candidates] +
                            [0 for _ in range(self.missed_true)]
                            )
        return gt, pred
    
SCENARIOS = {
    "Handpicked Hard": Scenario(0, 200, 800, 500, 500),
    "Handpicked Easy": Scenario(0, 800, 200, 500, 500),
    "Large selector": Scenario(5000, 800, 800, 500, 500),
    "Targeted selector": Scenario(50, 600, 800, 500, 500)
}

In [21]:
n_exp = 500

for sname, scenario in SCENARIOS.items():
    recs = []
    precs = []
    aps = []
    specs = []
    mccs = []
    
    print(sname)
    for _ in tqdm(range(n_exp)):
        # Compute single-threshold metrics
        gt, pred = scenario.random_draw()
        tp = (gt*(pred>=0.5)).sum()
        fp = ((gt==False)*(pred>=0.5)).sum()
        fn = (gt*(pred<0.5)).sum()
        tn = ((gt==False)*(pred<0.5)).sum()
        cm = np.array([
            [tn, fp],
            [fn, tp]
        ])
        ap, _ = average_precision(gt, pred)
        recs.append(recall(tp, fp, fn))
        precs.append(precision(tp, fp, fn))
        aps.append(ap)
        specs.append(SPEc(cm, 1))
        mccs.append(MCC(cm))
    
    print(f"REC={np.mean(recs):.2f} (std={np.std(recs):.3f})")
    print(f"PRE={np.mean(precs):.2f} (std={np.std(precs):.3f})")
    print(f"AP={np.mean(aps):.2f} (std={np.std(aps):.3f})")
    print(f"SPE={np.mean(specs):.2f} (std={np.std(specs):.3f})")
    print(f"MCC={np.mean(mccs):.2f} (std={np.std(mccs):.3f})")

Handpicked Hard


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:31<00:00, 15.78it/s]


REC=0.85 (std=0.010)
PRE=0.77 (std=0.010)
AP=0.92 (std=0.004)
SPE=0.75 (std=0.013)
MCC=0.60 (std=0.017)
Handpicked Easy


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:31<00:00, 15.73it/s]


REC=0.85 (std=0.011)
PRE=0.93 (std=0.007)
AP=0.98 (std=0.002)
SPE=0.94 (std=0.006)
MCC=0.79 (std=0.012)
Large selector


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:50<00:00,  9.93it/s]


REC=0.85 (std=0.010)
PRE=0.77 (std=0.009)
AP=0.92 (std=0.004)
SPE=0.96 (std=0.002)
MCC=0.78 (std=0.009)
Targeted selector


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:37<00:00, 13.29it/s]

REC=0.85 (std=0.011)
PRE=0.77 (std=0.010)
AP=0.92 (std=0.004)
SPE=0.83 (std=0.009)
MCC=0.67 (std=0.014)
